<a href="https://colab.research.google.com/github/nikhiltarte/TensorFlow-DeepLearning/blob/main/ML_Buzz_Beginer_Discount_based_on_customer_behavior.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import Libraries

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
from google.colab import files

Importing Dataset

In [2]:
#Import_Both Test and Train Dataset with name as train.csv and test.csv

uploaded = files.upload()

Saving test.csv to test.csv
Saving train.csv to train.csv


In [154]:
import io

train_Dataset = pd.read_csv(io.BytesIO(uploaded['train.csv']))
test_Dataset = pd.read_csv(io.BytesIO(uploaded['test.csv']))

train_Dataset

,Customer_ID,No_of_orders_placed,Category_of_customers,Maximum_bill,Minimum_bill,Last_order_placed_date,No_of_issues_raised,Customer_rating,Average_food_rating,Average_happiness_rating,Coupon_consumption_status,Coupons_offered,Premium_membership,Discount_percentage
0,ID0xb02f,129.000000,Active,2251.620000,261.26,04/02/2020 11:29 AM,129.000000,4,4,4,129.0,215.0,No,34
1,ID0x8cff,NaN,Inactive,-450.457485,98.63,02/22/2016 04:08 AM,NaN,4,4,2,118.0,246.0,No,42
2,ID0xc812,100.000000,Passive,1792.660000,144.00,08/23/2018 07:08 PM,100.000000,4,4,5,66.0,121.0,No,31
3,ID0xedcd,87.000000,Active,2260.730000,192.00,02/03/2020 06:31 AM,87.000000,4,4,3,59.0,118.0,No,19
4,ID0x93c3,40.000000,Passive,2083.890000,229.13,06/26/2019 01:30 PM,40.000000,4,5,2,40.0,50.0,No,21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53797,ID0x56a,250.000000,Active,3375.350000,148.97,11/17/2020 11:44 AM,250.000000,4,4,3,67.0,366.0,No,36
53798,ID0x82c8,226.000000,Passive,526.680000,208.94,11/18/2018 10:07 AM,226.000000,4,3,4,43.0,272.0,No,36
53799,ID0xafed,151.000000,Passive,3132.900000,264.12,12/28/2018 09:39 AM,151.000000,4,4,4,98.0,226.0,Yes,51
53800,ID0x9eff,230.000000,Inactive,1971.600000,101.73,05/29/2017 01:50 PM,230.000000,3,4,2,114.0,328.0,No,21


In [155]:
train_Dataset.dtypes



Customer_ID                   object
No_of_orders_placed          float64
Category_of_customers         object
Maximum_bill                 float64
Minimum_bill                 float64
Last_order_placed_date        object
No_of_issues_raised          float64
Customer_rating                int64
Average_food_rating            int64
Average_happiness_rating       int64
Coupon_consumption_status    float64
Coupons_offered              float64
Premium_membership            object
Discount_percentage            int64
dtype: object

In [156]:
#Replace NA By Mean
print("Train:\n==================================")
print(train_Dataset.isna().any(axis=0))

train_Dataset.isna().sum()
train_Dataset = train_Dataset.dropna()
#train_Dataset.fillna(train_Dataset.mean(), inplace=True)
test_Dataset.fillna(test_Dataset.mean(), inplace=True)

print("\n\nTrain:\n==================================")
print(train_Dataset.isna().any(axis=0))
print("\n\nTest:\n==================================")
print(train_Dataset.isna().any(axis=0))

Train:
Customer_ID                  False
No_of_orders_placed           True
Category_of_customers        False
Maximum_bill                 False
Minimum_bill                 False
Last_order_placed_date       False
No_of_issues_raised           True
Customer_rating              False
Average_food_rating          False
Average_happiness_rating     False
Coupon_consumption_status     True
Coupons_offered               True
Premium_membership           False
Discount_percentage          False
dtype: bool


Train:
Customer_ID                  False
No_of_orders_placed          False
Category_of_customers        False
Maximum_bill                 False
Minimum_bill                 False
Last_order_placed_date       False
No_of_issues_raised          False
Customer_rating              False
Average_food_rating          False
Average_happiness_rating     False
Coupon_consumption_status    False
Coupons_offered              False
Premium_membership           False
Discount_percentage        

In [157]:
#Drop Outliers for y
train_Dataset['Last_order_placed_date'] = pd.to_datetime(train_Dataset['Last_order_placed_date'], format='%m/%d/%Y %I:%M %p')
test_Dataset['Last_order_placed_date'] = pd.to_datetime(test_Dataset['Last_order_placed_date'], format='%m/%d/%Y %I:%M %p')

print(train_Dataset.dtypes)

Customer_ID                          object
No_of_orders_placed                 float64
Category_of_customers                object
Maximum_bill                        float64
Minimum_bill                        float64
Last_order_placed_date       datetime64[ns]
No_of_issues_raised                 float64
Customer_rating                       int64
Average_food_rating                   int64
Average_happiness_rating              int64
Coupon_consumption_status           float64
Coupons_offered                     float64
Premium_membership                   object
Discount_percentage                   int64
dtype: object


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [158]:
# Convert Datetime to Days from Max 
max_date_train = max(train_Dataset['Last_order_placed_date'])
train_Dataset['Last_order_placed_date'] = (max_date_train - train_Dataset['Last_order_placed_date']).dt.days
test_Dataset['Last_order_placed_date'] = (max_date_train - test_Dataset['Last_order_placed_date']).dt.days


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [142]:
#Negitive Dataset

train_Dataset.drop(train_Dataset.loc[(train_Dataset['Maximum_bill'] < 0) | (train_Dataset['Minimum_bill'] < 0) | (train_Dataset['No_of_orders_placed'] < 0) | (train_Dataset['No_of_issues_raised'] < 0)].index, inplace=True)
#test_Dataset.loc[(test_Dataset['Maximum_bill'] < 0) | (test_Dataset['Minimum_bill'] < 0)]


/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [159]:
train_Dataset

,Customer_ID,No_of_orders_placed,Category_of_customers,Maximum_bill,Minimum_bill,Last_order_placed_date,No_of_issues_raised,Customer_rating,Average_food_rating,Average_happiness_rating,Coupon_consumption_status,Coupons_offered,Premium_membership,Discount_percentage
0,ID0xb02f,129.0,Active,2251.62,261.26,332,129.0,4,4,4,129.0,215.0,No,34
2,ID0xc812,100.0,Passive,1792.66,144.00,920,100.0,4,4,5,66.0,121.0,No,31
3,ID0xedcd,87.0,Active,2260.73,192.00,391,87.0,4,4,3,59.0,118.0,No,19
4,ID0x93c3,40.0,Passive,2083.89,229.13,613,40.0,4,5,2,40.0,50.0,No,21
5,ID0xd263,158.0,Active,1629.92,256.96,289,158.0,4,4,4,113.0,205.0,No,45
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53796,ID0xa193,134.0,Passive,3180.09,319.21,837,134.0,4,4,4,45.0,222.0,No,36
53797,ID0x56a,250.0,Active,3375.35,148.97,103,250.0,4,4,3,67.0,366.0,No,36
53798,ID0x82c8,226.0,Passive,526.68,208.94,833,226.0,4,3,4,43.0,272.0,No,36
53799,ID0xafed,151.0,Passive,3132.90,264.12,793,151.0,4,4,4,98.0,226.0,Yes,51


In [160]:
#Analysing Dataset
train_Dataset.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
No_of_orders_placed,49737.0,127.368974,92.278305,-477.349415,62.00,127.00,192.00,724.669533
Maximum_bill,49737.0,1950.004482,1166.348828,-4946.845593,1115.55,1913.94,2724.53,10147.345902
Minimum_bill,49737.0,197.738539,82.317575,-304.917730,135.45,193.98,255.04,821.409409
Last_order_placed_date,49737.0,1063.767276,616.452181,0.000000,530.00,1063.00,1597.00,2130.000000
No_of_issues_raised,49737.0,127.368974,92.278305,-477.349415,62.00,127.00,192.00,724.669533
Customer_rating,49737.0,3.873394,0.643101,2.000000,4.00,4.00,4.00,5.000000
Average_food_rating,49737.0,3.696986,0.841274,1.000000,3.00,4.00,4.00,5.000000
Average_happiness_rating,49737.0,3.684199,0.848253,2.000000,3.00,4.00,4.00,5.000000
Coupon_consumption_status,49737.0,74.667390,42.231478,5.000000,39.00,69.00,106.00,170.000000
Coupons_offered,49737.0,184.394535,97.624393,15.000000,88.00,205.00,268.00,370.000000


In [161]:
train_Dataset.Category_of_customers = pd.Categorical(train_Dataset.Category_of_customers)
train_Dataset['Category_of_customers'] = train_Dataset.Category_of_customers.cat.codes

train_Dataset.Premium_membership = pd.Categorical(train_Dataset.Premium_membership)
train_Dataset['Premium_membership'] = train_Dataset.Premium_membership.cat.codes


test_Dataset.Category_of_customers = pd.Categorical(test_Dataset.Category_of_customers)
test_Dataset['Category_of_customers'] = test_Dataset.Category_of_customers.cat.codes

test_Dataset.Premium_membership = pd.Categorical(test_Dataset.Premium_membership)
test_Dataset['Premium_membership'] = test_Dataset.Premium_membership.cat.codes

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:5170: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.py

In [162]:
train_Dataset = train_Dataset.drop('Customer_ID', 1)


testCustermerID = test_Dataset['Customer_ID']
test_Dataset = test_Dataset.drop('Customer_ID', 1)

In [163]:
train_Dataset.shape

(49737, 13)

In [164]:
trainX = train_Dataset
trainX = trainX.drop('Discount_percentage',1)
trainY = train_Dataset['Discount_percentage']

testX = test_Dataset

In [ ]:
#from sklearn.model_selection import train_test_split
#X_train, X_test, y_train, y_test = train_test_split(trainX, trainY, test_size=0.20, random_state=33)

In [165]:
X_train = trainX
y_train = trainY

Model Training Start

In [166]:
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing

In [170]:
# create model
model = keras.Sequential()
model.add(keras.layers.Dense(50, input_dim=12, activation='relu'))
model.add(keras.layers.Dense(50, activation='relu'))
model.add(keras.layers.Dense(50, activation='relu'))
model.add(keras.layers.Dense(1))
# Compile model
opt = tf.keras.optimizers.RMSprop(0.0099)
model.compile(loss='mse', optimizer=opt)
# evaluate model with standardized dataset

history = model.fit(X_train,y_train, epochs=100)

Epoch 1/100
1555/1555 [==============================] - 6s 4ms/step - loss: 1217.0282 - mse: 1217.0282
Epoch 2/100
1555/1555 [==============================] - 5s 3ms/step - loss: 82.8109 - mse: 82.8109
Epoch 3/100
1555/1555 [==============================] - 5s 3ms/step - loss: 71.8305 - mse: 71.8305
Epoch 4/100
1555/1555 [==============================] - 5s 4ms/step - loss: 68.4866 - mse: 68.4866
Epoch 5/100
1555/1555 [==============================] - 5s 4ms/step - loss: 68.3548 - mse: 68.3548
Epoch 6/100
1555/1555 [==============================] - 5s 3ms/step - loss: 66.5362 - mse: 66.5362
Epoch 7/100
1555/1555 [==============================] - 5s 4ms/step - loss: 65.3753 - mse: 65.3753
Epoch 8/100
1555/1555 [==============================] - 6s 4ms/step - loss: 64.5178 - mse: 64.5178
Epoch 9/100
1555/1555 [==============================] - 5s 3ms/step - loss: 64.2419 - mse: 64.2419
Epoch 10/100
1555/1555 [==============================] - 5s 3ms/step - loss: 63.4583 - mse: 63.

In [1]:
y_predict = model.predict(X_train)
np.round(y_predict)

NameError: ignored

In [ ]:
from sklearn.metrics import r2_score
r2_score(y_train, y_predict)*100

In [ ]:
y_predict_test = model.predict(testX)
y_predict_test

In [ ]:
testCustermerID = pd.DataFrame(testCustermerID) 
y_predict_test = pd.DataFrame(y_predict_test) 

In [ ]:
submission = pd.DataFrame(testCustermerID)
submission['Discount_percentage'] = y_predict_test
submission

In [ ]:
submission.to_csv('Submission2.csv', index=False) 
files.download('Submission2.csv')